# Reproducing Hakim CDA Study (2019)
## Looking at Channels

In [2]:
import numpy as np
import glob
import pickle
import pandas as pd

import mne
from mne import Epochs, pick_channels, find_events
from mne.io import read_raw_brainvision

import sys
#sys.path.append('/Users/nick/Documents/PhD/CLCT/reprod/studies/')
sys.path.append('D:\\PhD\\CDA-review\\studies\\')
from cda_utils import get_epochs, get_CDA, prep_report, fill_report, get_report, checksum, add_cda_report, get_CDA_perf_report
#sys.path.append('/Users/nick/Documents/PhD/CLCT/reprod/studies/Hakim-2019/')
sys.path.append('D:\\PhD\\CDA-review\\studies\\Hakim-2019\\')
from events import get_specific_events
 
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Prep Data

In [3]:
#==================================================================
# General Variables
#==================================================================
study_id = 'Hakim2019'

triggers = dict();

triggers['2'] = [2, 3]
triggers['4'] = [4, 5]

triggers['left'] = [3, 5]   #[2, 4]
triggers['right'] = [2, 4]  #[3, 5]

triggers['good'] = [91]
triggers['bad'] = [90]

# Labels
conds = ['2', '4']
sides = ['left', 'right']
perfs = ['good', 'bad']

# Internal Triggers to Epochs Properly
internal_triggers = dict()
internal_triggers_count = 0
for cond in conds:
    for side in sides:
        for perf in perfs:
            internal_triggers_count = internal_triggers_count + 1
            internal_triggers['{}-{}-{}'.format(cond,side,perf)] = internal_triggers_count
            
# CDA Channels
chan_right = ['O2', 'PO4', 'PO8', 'P4', 'P8']
chan_left = ['O1', 'PO3', 'PO7', 'P3', 'P7']

# Epoch Information
epoch_length = 1.45
epoch_tmin = -0.4
baseline_corr = (-0.4, 0)

# CDA info
cda_window = (0.4, 1.45)

# CDA main variable!
cda_dict_all = dict()
epochs_dict_all = dict()
perf_report_all = dict()
report = prep_report(conds, sides, perfs)

In [2]:
#==================================================================
# Main Loop - Loading All Files & Extract CDAs.
#==================================================================
experiences = ['Exp1a', 'Exp1b', 'Exp2a', 'Exp2b'] 

eegfiles = []
for cur_exp in experiences:
    #folderpath = '/Volumes/Nick - Data/CDA Datasets/Hakim 2019/{}/EEGData/**/*.vhdr'.format(cur_exp)
    folderpath = 'G:\\CDA Datasets\\Hakim 2019\\{}\\EEGData\\**\\*.vhdr'.format(cur_exp)
    for filepath in glob.glob(folderpath, recursive=True):
        eegfiles.append(filepath)

sep = '\\'
for filepath in eegfiles:
    filename = filepath[filepath.rfind(sep)+1:]
    print('Processing: ' + filename + '...')
   
    # ------------------------
    # 1. -- Load File --
    # ------------------------
    raw = read_raw_brainvision(filepath, preload=True)

    # ------------------------
    # 2. -- Preprocessing --
    # ------------------------
    # Set Electrodes Locations - Used Brain Products (locations seem to in eeg file...)
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage)
    
    preproc = raw
    
    # Filter
    preproc = preproc.filter(.1, 20., fir_design='firwin', skip_by_annotation='edge')

    # Events
    events = mne.events_from_annotations(preproc)
        
    # Select Channels (to save time on cleaning)
    #preproc = preproc.pick_channels(chan_right + chan_left)
    
    # Plot (visual check)
    #preproc.plot(events=events, block=True, lowpass=40)
    
    # ------------------------
    # 3. -- Epochs --
    # ------------------------    
    specific_events = get_specific_events(events[0], conds, sides, perfs, triggers, internal_triggers)

    epochs_dict = get_epochs(preproc, specific_events, epoch_length, epoch_tmin, baseline_corr, clean=False)

    epochs_dict_all[filename] = epochs_dict
    
    # times, used for CDA mean amp report.
    times = epochs_dict[conds[0]][sides[0]][perfs[0]].times
    
    # ------------------------
    # 4. -- CDA --
    # ------------------------
    cda_dict_all[filename] = get_CDA(epochs_dict, conds, sides, perfs, chan_right, chan_left)
    
    # ------------------------
    # 6. - Report & Checksum -
    # ------------------------
    report = fill_report(report, filename, specific_events)
    report = add_cda_report(report, filename, conds, sides, perfs, cda_dict_all[filename], times, cda_window=cda_window)
    report.at[filename, 'checksum'] = checksum(specific_events, epochs_dict)
    
    perf_report_all[filename] = get_CDA_perf_report(epochs_dict, conds, sides, perfs, chan_right, chan_left, cda_window=cda_window)

# Print report
print('{} Subjects & {} Trials'.format(len(report), 2*np.sum([report[cond].sum() for cond in conds])))
report

Processing: 10_WM_att_NHKA.vhdr...
Extracting parameters from G:\CDA Datasets\Hakim 2019\Exp1a\EEGData\10\10_WM_att_NHKA.vhdr...
Setting channel info structure...
Reading 0 ... 5127809  =      0.000 ... 10255.618 secs...


<ipython-input-2-2830dbe7c349>:69: RuntimeWarning: No coordinate information found for channels ['HEOG', 'VEOG', 'StimTrak']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = read_raw_brainvision(filepath, preload=True)
<ipython-input-2-2830dbe7c349>:69: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = read_raw_brainvision(filepath, preload=True)
<ipython-input-2-2830dbe7c349>:69: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw = read_raw_brainvision(filepath, preload=True)
<ipython-input-2-2830dbe7c349>:69: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (250.00 Hz, Nyquist limit) will be stored.
  raw = read_raw_brainvision(filepath, preload=True)
<ipython-input-2-2830dbe7c349>:69: RuntimeWarning: Not setting positions of 3 misc channels found in montage:

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 20 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 22.50 Hz)
- Filter length: 16501 samples (33.002 sec)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S 12', 'Stimulus/S 13', 'Stimulus/S 14', 'Stimulus/S 15', 'Stimulus/S 90', 'Stimulus/S 91', 'Stimulus/S100', 'Stimulus/S101', 'Stimulus/S102', 'Stimulus/S103', 'Stimulus/S104', 'Stimulus/S105', 'Stimulus/S106']
A total of 800 events were added and 0 were skipped.
====================== EPOCHING ===========

D:\PhD\CDA-review\studies\cda_utils.py:40: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  all_epochs = Epochs(eeg, all_events, tmin=epoch_tmin, tmax=epoch_length, baseline=baseline_corr, preload=True, event_repeated='merge')


0 bad epochs dropped
A total of 800 epochs were extracted and baseline corrected.
---------- 2-left-good -------------
---------- 2-left-bad -------------
---------- 2-right-good -------------
---------- 2-right-bad -------------
---------- 4-left-good -------------
---------- 4-left-bad -------------
---------- 4-right-good -------------
---------- 4-right-bad -------------
====================== /EPOCHING ======================
[10_WM_att_NHKA.vhdr] Adding Mean CDA Amp for 2-left-good: -0.5760541245343758
[10_WM_att_NHKA.vhdr] Adding Mean CDA Amp for 2-left-bad: -0.4592184513903521
[10_WM_att_NHKA.vhdr] Adding Mean CDA Amp for 2-right-good: 2.479374131750155
[10_WM_att_NHKA.vhdr] Adding Mean CDA Amp for 2-right-bad: 3.1239862571956216
[10_WM_att_NHKA.vhdr] Adding Mean CDA Amp for 4-left-good: 0.09910275283988186
[10_WM_att_NHKA.vhdr] Adding Mean CDA Amp for 4-left-bad: -0.09057697883890312
[10_WM_att_NHKA.vhdr] Adding Mean CDA Amp for 4-right-good: 2.231749496458544
[10_WM_att_NHKA.v

...

<ipython-input-2-2830dbe7c349>:69: RuntimeWarning: No coordinate information found for channels ['HEOG', 'VEOG', 'StimTrak']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = read_raw_brainvision(filepath, preload=True)
<ipython-input-2-2830dbe7c349>:69: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = read_raw_brainvision(filepath, preload=True)
<ipython-input-2-2830dbe7c349>:69: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw = read_raw_brainvision(filepath, preload=True)
<ipython-input-2-2830dbe7c349>:69: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (250.00 Hz, Nyquist limit) will be stored.
  raw = read_raw_brainvision(filepath, preload=True)
<ipython-input-2-2830dbe7c349>:69: RuntimeWarning: Not setting positions of 3 misc channels found in montage:

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 20 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 22.50 Hz)
- Filter length: 16501 samples (33.002 sec)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S 12', 'Stimulus/S 13', 'Stimulus/S 14', 'Stimulus/S 15', 'Stimulus/S 90', 'Stimulus/S 91', 'Stimulus/S100', 'Stimulus/S101', 'Stimulus/S102', 'Stimulus/S103', 'Stimulus/S104', 'Stimulus/S105', 'Stimulus/S106']
A total of 800 events were added and 0 were skipped.
====================== EPOCHING ===========

D:\PhD\CDA-review\studies\cda_utils.py:40: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  all_epochs = Epochs(eeg, all_events, tmin=epoch_tmin, tmax=epoch_length, baseline=baseline_corr, preload=True, event_repeated='merge')


0 bad epochs dropped
A total of 800 epochs were extracted and baseline corrected.
---------- 2-left-good -------------
---------- 2-left-bad -------------
---------- 2-right-good -------------
---------- 2-right-bad -------------
---------- 4-left-good -------------
---------- 4-left-bad -------------
---------- 4-right-good -------------
---------- 4-right-bad -------------
====================== /EPOCHING ======================
[8_KiNi4.vhdr] Adding Mean CDA Amp for 2-left-good: 0.22069924263910257
[8_KiNi4.vhdr] Adding Mean CDA Amp for 2-left-bad: 0.12122663487868755
[8_KiNi4.vhdr] Adding Mean CDA Amp for 2-right-good: -0.29817100525832085
[8_KiNi4.vhdr] Adding Mean CDA Amp for 2-right-bad: -0.9564258554665876
[8_KiNi4.vhdr] Adding Mean CDA Amp for 4-left-good: 0.9923760103198892
[8_KiNi4.vhdr] Adding Mean CDA Amp for 4-left-bad: 0.7551466940751855
[8_KiNi4.vhdr] Adding Mean CDA Amp for 4-right-good: 0.04273180130380582
[8_KiNi4.vhdr] Adding Mean CDA Amp for 4-right-bad: -0.30921253

,2,4,left,right,good,bad,2-left,2-left-CDA,2-left-good,2-left-good-CDA,...,4-right-good,4-right-good-CDA,right-good-CDA,4-right-bad,4-right-bad-CDA,right-bad-CDA,checksum,notes,good-CDA,bad-CDA
10_WM_att_NHKA.vhdr,400,400,400,400,571,229,200,0,154,-0.576054,...,128,2.231749,2.355562,72,0.991957,2.057972,1,,1.058543,0.891537
11_WM_att_NHKA.vhdr,400,400,400,400,665,135,200,0,190,1.123072,...,154,-0.128962,-0.149271,46,-0.069433,0.12858,1,0,0.540559,0.750825
12_WM_att_NHKA.vhdr,400,400,400,400,723,77,200,0,192,1.499339,...,168,0.648914,0.517893,32,0.391398,0.329335,1,0,0.980741,1.021325
13_WM_att_NHKA.vhdr,400,400,400,400,620,180,200,0,169,0.819664,...,136,-0.14181,-0.37906,64,-0.850073,-0.943938,1,0,0.284688,-0.424776
15_WM_att_NHKA.vhdr,400,400,400,400,484,316,200,0,140,-0.255138,...,103,0.47914,0.877938,97,1.115119,0.786751,1,0,0.311437,0.02933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5_KiNi4.vhdr,400,400,400,400,662,138,200,0,174,1.673581,...,153,-0.148444,-0.429331,47,-0.135827,-0.466873,1,0,0.567079,0.462775
50_KiNi4.vhdr,400,400,400,400,646,154,200,0,175,2.710157,...,152,0.926929,0.346854,48,0.587205,0.480709,1,0,1.516376,1.448504
6_KiNi4.vhdr,400,400,400,400,598,202,200,0,163,0.338282,...,138,0.197559,0.444655,62,0.348664,0.296188,1,0,0.247470,0.035646
7_KiNi4.vhdr,400,400,400,400,421,379,200,0,112,1.262822,...,91,2.045883,1.708287,109,1.6817,1.628652,1,0,1.355393,1.364327


In [4]:
with open('D:\\PhD\\CDA-review\\studies\\Hakim-2019-Channels.pkl', 'wb') as f:
    pickle.dump(epochs_dict_all, f)
    
del epochs_dict_all

In [5]:
with open('D:\\PhD\\CDA-review\\studies\\_channels\\Hakim-2019-Channels.pkl', 'rb') as f:
    epochs_dict_all = pickle.load(f)

In [8]:
del epochs_dict_all

In [7]:
# YR: Confirmed channels!
chan_right_all = ['Fp2', 'F4', 'F8', 'CP6', 'FC6', 'FC2', 'C4', 'PO4', 'PO8', 'CP2', 'P4', 'P8', 'O2']
chan_left_all =  ['Fp1', 'F3', 'F7', 'CP5', 'FC5', 'FC1', 'C3', 'PO3', 'PO7', 'CP1', 'P3', 'P7', 'O1']

cda_dict = dict()
cda_all_channels = dict()
for cond in conds:
    cda_all_channels[cond] = dict()
    for perf in perfs:
        cda_all_channels[cond][perf] = dict()
    
for i, ch in enumerate(chan_right_all):
    for file_name in epochs_dict_all.keys():
        epochs_dict = epochs_dict_all[file_name]
        cda_dict[file_name] = dict()

        times = epochs_dict[conds[0]][sides[0]][perfs[0]].times

        for cur_cond in conds:
            cda_dict[file_name][cur_cond] = dict()

            for cur_side in sides:
                cda_dict[file_name][cur_cond][cur_side] = dict()

                for cur_perf in ['good']:#perfs:
                    if cur_perf in epochs_dict[cur_cond][cur_side].keys():
                        if epochs_dict[cur_cond][cur_side][cur_perf] is not None:
                            epochs_right = epochs_dict[cur_cond][cur_side][cur_perf].copy().pick_channels([chan_right_all[i]])
                            epochs_left = epochs_dict[cur_cond][cur_side][cur_perf].copy().pick_channels([chan_left_all[i]])

                            evoked_right = epochs_right.average()
                            evoked_left = epochs_left.average()

                            left = evoked_left.data.mean(0)
                            right = evoked_right.data.mean(0)

                            if cur_side == 'left':
                                cda_dict[file_name][cur_cond][cur_side][cur_perf] = left - right
                            else:
                                cda_dict[file_name][cur_cond][cur_side][cur_perf] = right - left

    cda_all = dict()

    for cond in cda_dict[list(cda_dict.keys())[0]].keys():
        cda_all[cond] = dict()
        
        for perf in perfs:
            cda_all[cond][perf] = dict()

            for f in cda_dict.keys():
                if (perf in cda_dict[f][cond]['left'].keys()) and (perf in cda_dict[f][cond]['right'].keys()):
                    if len(cda_dict[f][cond]['left'][perf]) > 0:
                        cda_dict_cur = (cda_dict[f][cond]['left'][perf] + cda_dict[f][cond]['right'][perf])/2
                        cda_all[cond][perf] = np.vstack([cda_all[cond][perf], cda_dict_cur]) if len(cda_all[cond][perf]) else cda_dict_cur
                    else:
                        print("==== SKIPPED: 0 points! (File: {} - Cond: {} - Perf: {})".format(str(f), str(cond), perf))
        
            cda_all_channels[cond][perf]["{}-{}".format(chan_left_all[i], chan_right_all[i])] = cda_all[cond][perf].mean(0)

AttributeError: 'dict' object has no attribute 'mean'

In [1]:
perfs

NameError: name 'perfs' is not defined

In [ ]:
times = epochs_dict_all[filename][cond]['left'][perf].times

In [ ]:
cmap = matplotlib.cm.get_cmap('YlOrBr')

for perf in perfs:
    for cond in conds:
        plt.figure(figsize=(15, 12), dpi=80)
        plt.suptitle("Cond: {} [{}] - All Channel Pairs".format(cond, perf), fontsize=24)

        ch_pair_mean_amp = [cda_all_channels[cond][perf][x].mean(0) for x in cda_all_channels[cond][perf]]
        mean_list, cda_sorted = zip(*sorted(zip(ch_pair_mean_amp, cda_all_channels[cond][perf].items()), reverse=True))

        nb_pairs = len(cda_all_channels[cond][perf].keys())
        for i, ch_pair in enumerate(cda_all_channels[cond][perf].keys()):
            plt.plot((times * 1000), cda_sorted[i][1], label=cda_sorted[i][0], color=cmap(1-i/nb_pairs))

        plt.axvline(x=0, color='k', linestyle='--')
        plt.axvspan(600, 700, alpha=0.9, facecolor='lightgray')
        #plt.gca().invert_yaxis()
        plt.legend()
        plt.show()